# Day 16: Reindeer Maze
It's time again for the Reindeer Olympics! This year, the big event is the Reindeer Maze, where the Reindeer compete for the lowest score.

You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?

The Reindeer start on the Start Tile (marked S) facing East and need to reach the End Tile (marked E). They can move forward one tile at a time (increasing their score by 1 point), but never into a wall (#). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by 1000 points).

To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:

```
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
```

There are many paths through this maze, but taking any of the best paths would incur a score of only 7036. This can be achieved by taking a total of 36 steps forward and turning 90 degrees a total of 7 times:

```
###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#..>>>>>>>>v#^#
###^#.#####v#^#
#>>^#.....#v#^#
#^#.#.###.#v#^#
#^....#...#v#^#
#^###.#.#.#v#^#
#S..#.....#>>^#
###############
```

Here's a second example:

```
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
```

In this maze, the best paths cost 11048 points; following one such path would look like this:

```
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#^#
#.#.#.#...#...#^#
#.#.#.#.###.#.#^#
#>>v#.#.#.....#^#
#^#v#.#.#.#####^#
#^#v..#.#.#>>>>^#
#^#v#####.#^###.#
#^#v#..>>>>^#...#
#^#v###^#####.###
#^#v#>>^#.....#.#
#^#v#^#####.###.#
#^#v#^........#.#
#^#v#^#########.#
#S#>>^..........#
#################
```

Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.

Analyze your map carefully. What is the lowest score a Reindeer could possibly get?

In [3]:
import networkx as nx

def v_north(y,x):
    return "north_" + str(y) + "_" + str(x)

def v_south(y,x):
    return "south_" + str(y) + "_" + str(x)

def v_east(y,x):
    return "east_" + str(y) + "_" + str(x)

def v_west(y,x):
    return "west_" + str(y) + "_" + str(x)

def result(inputfile):
    map = []
    with open(inputfile) as f:
        for line in f:
            map.append([*line.strip()])

    G = nx.Graph()

    for y, line in enumerate(map):
        for x, char in enumerate(line):
            if char != "#":
                G.add_edge(v_north(y,x), v_west(y,x), weight=1000)
                G.add_edge(v_west(y,x), v_south(y,x), weight=1000)
                G.add_edge(v_south(y,x), v_east(y,x), weight=1000)
                G.add_edge(v_east(y,x), v_north(y,x), weight=1000)
                if not map[y - 1][x] == "#":
                    G.add_edge(v_north(y,x), v_north(y - 1,x), weight=1)
                if not map[y + 1][x] == "#":
                    G.add_edge(v_south(y,x), v_south(y + 1,x), weight=1)
                if not map[y][x - 1] == "#":
                    G.add_edge(v_west(y,x), v_west(y,x - 1), weight=1)
                if not map[y][x + 1] == "#":
                    G.add_edge(v_east(y,x), v_east(y,x + 1), weight=1)
            if char == "S":
                start_x = x
                start_y = y
            if char == "E":
                end_x = x
                end_y = y
        
    return min(nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_east(end_y, end_x), weight="weight"), \
       nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_west(end_y, end_x), weight="weight"), \
       nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_south(end_y, end_x), weight="weight"), \
       nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_north(end_y, end_x), weight="weight"))
                

In [4]:
result("example-input.txt")

7036

In [5]:
result("example-input-2.txt")

11048

In [6]:
result("input.txt")

160624

# Part Two
Now that you know what the best paths look like, you can figure out the best spot to sit.

Every non-wall tile (S, ., or E) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is whether the tile is on one of the best paths through the maze. If you sit somewhere else, you'd miss all the action!

So, you'll need to determine which tiles are part of any best path through the maze, including the S and E tiles.

In the first example, there are 45 tiles (marked O) that are part of at least one of the various best paths through the maze:

```
###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
```

In the second example, there are 64 tiles that are part of at least one of the best paths:

```
#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################
```

Analyze your map further. How many tiles are part of at least one of the best paths through the maze?

In [18]:
from collections import defaultdict

def result2(inputfile):
    map = []
    with open(inputfile) as f:
        for line in f:
            map.append([*line.strip()])

    G = nx.DiGraph()

    for y, line in enumerate(map):
        for x, char in enumerate(line):
            if char != "#":
                G.add_edge(v_north(y,x), v_west(y,x), weight=1000)
                G.add_edge(v_west(y,x), v_south(y,x), weight=1000)
                G.add_edge(v_south(y,x), v_east(y,x), weight=1000)
                G.add_edge(v_east(y,x), v_north(y,x), weight=1000)

                G.add_edge(v_north(y,x), v_east(y,x), weight=1000)
                G.add_edge(v_east(y,x), v_south(y,x), weight=1000)
                G.add_edge(v_south(y,x), v_west(y,x), weight=1000)
                G.add_edge(v_west(y,x), v_north(y,x), weight=1000)
                if not map[y - 1][x] == "#":
                    G.add_edge(v_north(y,x), v_north(y - 1,x), weight=1)
                if not map[y + 1][x] == "#":
                    G.add_edge(v_south(y,x), v_south(y + 1,x), weight=1)
                if not map[y][x - 1] == "#":
                    G.add_edge(v_west(y,x), v_west(y,x - 1), weight=1)
                if not map[y][x + 1] == "#":
                    G.add_edge(v_east(y,x), v_east(y,x + 1), weight=1)
            if char == "S":
                start_x = x
                start_y = y
            if char == "E":
                end_x = x
                end_y = y
        
    toEast = nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_east(end_y, end_x), weight="weight")
    toWest = nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_west(end_y, end_x), weight="weight")
    toSouth = nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_south(end_y, end_x), weight="weight")
    toNorth = nx.shortest_path_length(G, source=v_east(start_y, start_x), target=v_north(end_y, end_x), weight="weight")

    minLen = min(toEast, toNorth, toSouth, toWest)

    result_map = defaultdict(lambda: False)

    if toEast == minLen:
        all_shortest = list(nx.all_shortest_paths(G, source=v_east(start_y, start_x), target=v_east(end_y, end_x), weight="weight"))
        for path in all_shortest:
            for node in path:
                result_map[node] = True
    if toNorth == minLen:
        all_shortest = list(nx.all_shortest_paths(G, source=v_east(start_y, start_x), target=v_north(end_y, end_x), weight="weight"))
        for path in all_shortest:
            for node in path:
                result_map[node] = True
    if toSouth == minLen:
        all_shortest = list(nx.all_shortest_paths(G, source=v_east(start_y, start_x), target=v_south(end_y, end_x), weight="weight"))
        for path in all_shortest:
            for node in path:
                result_map[node] = True
    if toWest == minLen:
        all_shortest = list(nx.all_shortest_paths(G, source=v_east(start_y, start_x), target=v_west(end_y, end_x), weight="weight"))
        for path in all_shortest:
            for node in path:
                result_map[node] = True

    result = 0
    for y in range(len(map)):
        for x in range(len(map[0])):
            if result_map[v_east(y,x)] or result_map[v_north(y,x)] or \
               result_map[v_south(y,x)] or result_map[v_west(y,x)]:
                    result += 1
    
    return result

In [19]:
result2("example-input.txt")

45

In [20]:
result2("example-input-2.txt")

64

In [21]:
result2("input.txt")

692